# <h1 style="color:red;">Initial Setup</h>

Import all necessary modules

In [ ]:
import nest_asyncio
import asyncio
import schwabdev
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time
import os
import json
import pandas as pd
import pytz
import numpy as np
import mplfinance as mpf
from IPython.display import display, clear_output
import warnings
from Multi_Strat_Back_Tester_Functions import *
from datetime import datetime, time
import matplotlib.pyplot as plt
from itertools import product

Load environment variables from .env file for authentification purposes

In [ ]:
load_dotenv()

True

Patches the notebook's running event loop so that it can handle multiple calls to `asyncio.run()` or other asynchronous methods without causing conflicts

In [ ]:
nest_asyncio.apply()

Set the display options

In [ ]:
# Display options
# Set the global float format to 4 decimal places
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
pd.options.display.float_format = '{:,.4f}'.format # Format numerical output to have certain number of decimals
# pd.options.display.float_format = None # Reset to default numerical output formatting
pd.set_option('display.width', 2000) # Set the display width to a large number
pd.set_option('display.max_colwidth', 1000) # Set max column width to a large number

pd.set_option('display.max_columns', None) # Displays all columns
# pd.set_option('display.max_rows', None) # Displays all rows                             
# pd.reset_option('display.max_columns') # Display default abbreviated columns
pd.reset_option('display.max_rows') # Display default abbreviated rows

Create the client object.

In [ ]:
app_key = os.getenv('app_key')
app_secret = os.getenv('app_secret')
callback_url = os.getenv('callback_url')

# Print them to verify (avoid printing sensitive info in production)
print(f"App Key: {app_key}")
print(f"App Secret: {app_secret}")
print(f"Callback URL: {callback_url}")

# Now proceed to initialize the client
client = schwabdev.Client(app_key, app_secret, callback_url)
print('Client initialized!')

App Key: 2C7Jh6zt5QdlSb0N5RnhWpaiEzKFr150
App Secret: OQGxH0GkD5bAEMA4
Callback URL: https://127.0.0.1
The refresh token has expired, please update!
Open to authenticate: https://api.schwabapi.com/v1/oauth/authorize?client_id=2C7Jh6zt5QdlSb0N5RnhWpaiEzKFr150&redirect_uri=https://127.0.0.1
Client initialized!


Use this when in need of updating the refresh token.

In [ ]:
# client.update_tokens(force=True)
# client = schwabdev.Client(app_key, app_secret, callback_url)

# <h1 style="color:red;">Permanent Data Extraction and Storage Functions</h>

Ticker list and point value definition.

In [ ]:
tickers = "/ES,/NQ,/CL,/GC"
# tickers = "MARA,PLUG,SOFI,SWN,RKLB"

ticker_to_point_value = {
    "/ES": 50,       # E-Mini S&P 500
    "/NQ": 20,       # E-Mini NASDAQ 100
    "/CL": 1000,     # Crude Oil
    "/GC": 100,      # Gold
}

ticker_to_tick_size = {
    "/ES": 0.25,        # E-Mini S&P 500
    "/NQ": 0.25,        # E-Mini NASDAQ 100
    "/CL": 0.01,        # Crude Oil
    "/GC": 0.1,         # Gold
}

tick_size = ticker_to_tick_size.get("/ES")
tick_size

0.25

In [ ]:
# tickers = "/ES,/NQ,/CL,/GC,/MES,/MNQ,/MCL,/MGC"
# # tickers = "MARA,PLUG,SOFI,SWN,RKLB"

# ticker_to_point_value = {
#     "/ES": 50,       # E-Mini S&P 500
#     "/NQ": 20,       # E-Mini NASDAQ 100
#     "/CL": 1000,     # Crude Oil
#     "/GC": 100,      # Gold
#     "/MES": 5,       # Micro E-Mini S&P 500
#     "/MNQ": 2,       # Micro E-Mini NASDAQ 100
#     "/MCL": 100,     # Micro Crude Oil
#     "/MGC": 10       # Micro Gold
# }

# ticker_to_tick_size = {
#     "/ES": 0.25,        # E-Mini S&P 500
#     "/NQ": 0.25,        # E-Mini NASDAQ 100
#     "/CL": 0.01,        # Crude Oil
#     "/GC": 0.1,         # Gold
#     "/MES": 0.25,       # Micro E-Mini S&P 500
#     "/MNQ": 0.25,       # Micro E-Mini NASDAQ 100
#     "/MCL": 0.01,       # Micro Crude Oil
#     "/MGC": 0.1         # Micro Gold
# }

# tick_size = ticker_to_tick_size.get("/ES")
# tick_size

Extracting and storing raw streaming data with two functions into ticker_tables dictionary. 

In [ ]:
def extract_row_data(entry, content):
    
    # Create a row dictionary with the relevant fields
    row = { # For futures
        'timestamp': entry['timestamp'],
        'key': content.get('key', None),
        'bid_price': content.get('1', None),
        'ask_price': content.get('2', None),
        # 'last_price': last_price,  # Use updated last_price
        'last_price': content.get('3', None),
        'bid_size': content.get('4', None), # These two variables can be correlated 
        'ask_size': content.get('5', None), # with direction of price action
        'total_volume': content.get('8', None),
        'last_size': content.get('9', 0),
        'high_price': content.get('12', None),
        'low_price': content.get('13', None),
        'close_price': content.get('14', None),
        'open_price': content.get('18', None),
        'net_change': content.get('19', None),
        'future_pct_change': content.get('20', None),
        'open_interest': content.get('23', None),
        'tick': content.get('25', None),
        'tick_amount': content.get('26', None),
        'future_exp_date': content.get('35', None),
        'ask_time': content.get('37', None),
        'bid_time': content.get('38', None)
        }

    # row = { # For equities
    #     'timestamp': entry['timestamp'],               # Same
    #     'key': content.get('key', None),               # Same
    #     'bid_price': content.get('1', None),           # Matches bid_price
    #     'ask_price': content.get('2', None),           # Matches ask_price
    #     'last_price': last_price,                      # Matches last_price (updated)
    #     'bid_size': content.get('4', None),            # Matches bid_size
    #     'ask_size': content.get('5', None),            # Matches ask_size
    #     'total_volume': content.get('8', None),        # Matches total_volume
    #     'last_size': content.get('9', 0),              # Matches last_size
    #     'high_price': content.get('10', None),         # Matches high_price
    #     'low_price': content.get('11', None),          # Matches low_price
    #     'close_price': content.get('12', None),        # Matches close_price
    #     'open_price': content.get('17', None),         # Matches open_price
    #     'net_change': content.get('18', None),         # Matches net_change
    #     '52_week_high': content.get('19', None),       # Matches 52_week_high
    #     '52_week_low': content.get('20', None),        # Matches 52_week_low
    #     'pe_ratio': content.get('21', None),           # Matches pe_ratio
    #     'net_pct_change': content.get('42', None)}      # Matches net_pct_change  
    
    return row


In [ ]:
ticker_tables = {ticker: pd.DataFrame() for ticker in tickers.split(",")}

# Initialize a dictionary to store the last known price for each ticker
last_known_price = {ticker: None for ticker in tickers.split(",")}

def process_message_data(data):
    global ticker_tables, last_known_price
    
    # Loop through the content and extract the relevant fields
    if "data" in data:
        for entry in data['data']:
            if 'content' in entry:
                for content in entry['content']:
                    # Extract row data
                    row = extract_row_data(entry, content)

                    # Get the ticker (key) and last_price
                    ticker = row['key']
                    last_price = row.get('last_price')

                    # Check if last_price is None, if so use the last known price for the ticker
                    if last_price is None and ticker in last_known_price:
                        row['last_price'] = last_known_price[ticker]
                    else:
                        # Update the last known price
                        last_known_price[ticker] = last_price

                    # Convert the row dictionary to a DataFrame
                    row_df = pd.DataFrame([row])

                    # Check the ticker and append the row to the correct ticker table
                    if ticker in ticker_tables:
                        ticker_tables[ticker] = pd.concat([ticker_tables[ticker], row_df], ignore_index=True)

Compressing minute candles from the raw data.

In [ ]:
minute_candles = 'nothing'
minute_candles = {ticker: pd.DataFrame(columns=['datetime', 'ticker', 'open', 'high', 'low', 'close', 'accumulative_volume']) for ticker in tickers.split(",")}

def update_minute_candles(ticker_tables, time_frame='min'):
    global minute_candles

    for ticker, df in ticker_tables.items():
        if df.empty:
            continue
        
        # Ensure that 'datetime' column exists and is properly set
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True).dt.tz_convert('US/Eastern')

        # Ensure that the 'datetime' column is set as the DataFrame index (only if not already set)
        if df.index.name != 'datetime':
            df.set_index('datetime', inplace=True)

        # Group the data by minute intervals using 'min'
        current_minute = df.index.floor(time_frame)[-1]  # Get the most recent minute

        # Select only the rows from the most recent minute
        recent_data = df[df.index.floor(time_frame) == current_minute]

        if not recent_data.empty:
            # Extract OHLC values for the minute candle
            open_price = recent_data['last_price'].iloc[0]  # Open is the first last_price
            high_price = recent_data['last_price'].max()     # High is the max last_price
            low_price = recent_data['last_price'].min()      # Low is the min last_price
            close_price = recent_data['last_price'].iloc[-1] # Close is the last last_price
            total_volume = recent_data['total_volume'].max() # Total volume is the max value in this interval

            # Create a dictionary with the new candle data
            new_candle = {
                'datetime': current_minute,
                'ticker': ticker,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'accumulative_volume': total_volume
            }

            # Convert the dictionary to a DataFrame and update the respective ticker's DataFrame in minute_candles
            new_candle_df = pd.DataFrame([new_candle])

            # Replace or append the most recent candle for this ticker
            if not minute_candles[ticker].empty and minute_candles[ticker]['datetime'].iloc[-1] == current_minute:
                minute_candles[ticker].iloc[-1] = new_candle  # Update the last candle
            else:
                minute_candles[ticker] = pd.concat([minute_candles[ticker], new_candle_df], ignore_index=True)  # Append new candle

Defining a function to calculate a price action envelope indicator for trend determination.

In [ ]:
def calculate_price_action_envelope(df, high_col='high', low_col='low', window=5):
    """
    Calculate the continuous higher highs, higher lows, lower highs, and lower lows
    to create an envelope that wraps around the price action.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the price data.
    high_col (str): The column name for the high prices (default 'high').
    low_col (str): The column name for the low prices (default 'low').
    window (int): The window size for detecting local highs and lows (default 5).
    
    Returns:
    pd.DataFrame: The DataFrame with added columns for the price action envelope.
    """
    # Calculate rolling max (higher highs) and rolling min (lower lows)
    df['higher_high'] = df[high_col].rolling(window=window, min_periods=1).max()
    df['lower_low'] = df[low_col].rolling(window=window, min_periods=1).min()
    
    # Calculate rolling min of high for higher lows, and rolling max of low for lower highs
    df['higher_low'] = df[low_col].rolling(window=window, min_periods=1).min().cummax()
    df['lower_high'] = df[high_col].rolling(window=window, min_periods=1).max().cummin()
    
    # Create the envelope by combining these calculated values
    df['price_action_upper'] = df[['higher_high', 'lower_high']].max(axis=1)
    df['price_action_lower'] = df[['lower_low', 'higher_low']].min(axis=1)
    
    # Drop intermediate columns if not needed
    df.drop(columns=['higher_high', 'lower_low', 'higher_low', 'lower_high'], inplace=True)

    # Add moving averages for price_action_upper and price_action_lower
    ma_period = window  # Set the moving average period length to 5
    df['ma_price_action_upper'] = df['price_action_upper'].rolling(window=ma_period, min_periods=1).mean()
    df['ma_price_action_lower'] = df['price_action_lower'].rolling(window=ma_period, min_periods=1).mean()

    # Define a function to calculate trend score for an entire column over a 5-period window
    def calculate_trend_score(col_series):
        score_series = []
        for i in range(len(col_series)):
            score = 0
            for j in range(1, window):
                if i - j >= 0:
                    if col_series.iloc[i] > col_series.iloc[i - j]:
                        score += 10
                    elif col_series.iloc[i] < col_series.iloc[i - j]:
                        score -= 10
            score_series.append(score)
        return score_series

    # Calculate the trend indicator score for each relevant column and combine them
    scores_upper = calculate_trend_score(df['price_action_upper'])
    scores_lower = calculate_trend_score(df['price_action_lower'])

    # Sum the scores for the final trend indicator based only on price_action_upper and price_action_lower
    total_score = [u + l for u, l in zip(scores_upper, scores_lower)]

    # Rescale total_score to a 0-100 range and assign to 'trend_indicator'
    df['trend_indicator'] = [(score + 100) / 2 for score in total_score]

    return df

A function that calculates a number of indicators.

In [ ]:
def calculate_indicators(df, price_col='close', acc_vol_col='accumulative_volume', sma_periods=None, wma_periods=None, rsi_periods=None, volume_col='volume', candle_window=10):
    """
    Calculate SMAs and WMAs for given periods and append them to the DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the price data.
    price_col (str): The column name for the price data (default 'close').
    acc_vol_col (str): The column name for the accumulative volume data (default 'accumulative_volume').
    sma_periods (list): A list of periods for SMAs (e.g., [3, 5, 10, 20]).
    wma_periods (list): A list of periods for WMAs (e.g., [3, 5, 10, 20]).
    volume_col (str): The column name to store the volume difference (default 'volume').
    
    Returns:
    pd.DataFrame: The DataFrame with the calculated SMAs and WMAs appended.
    """
    # Calculate volume differences
    df[volume_col] = df[acc_vol_col].diff()
    
    # Calculate SMAs
    if sma_periods:
        for period in sma_periods:
            sma_col_name = f'sma_{period}'
            df[sma_col_name] = df[price_col].rolling(window=period).mean()
    
    # Calculate WMAs
    if wma_periods:
        for period in wma_periods:
            weights = np.arange(1, period + 1)  # Generate weights from 1 to the period length
            wma_col_name = f'wma_{period}'
            df[wma_col_name] = df[price_col].rolling(window=period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)

    # calculate_vwap_and_bands(df, high_col='high', low_col='low', close_col='close', volume_col='volume')

    # Assume 'periods' is a predefined list of different periods for which you want to calculate RSI
    for period in rsi_periods:
        # Calculate RSI for each period
        delta = df[price_col].diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))

    # Calculate price action envelope
    df = calculate_price_action_envelope(df, high_col='high', low_col='low', window=5)

    df['ohlc_average'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
    df['candle_span'] = df['high'] - df['low']
    df['candle_body'] = (df['close'] - df['open']).abs()
    df['candle_span_avg'] = df['candle_span'].rolling(window=candle_window, min_periods=1).mean()
    df['candle_span_max'] = df['candle_span'].rolling(window=candle_window, min_periods=1).max()
    df['candle_span_maxavg_mean'] =(df['candle_span_avg'] + df['candle_span_max'])/2
    df['hundred_line'] = 100
    df['fifty_line'] = 50
    df['zero_line'] = 0
    df['trend_high_threshold'] = 75
    df['trend_low_threshold'] = 25

    return df

# <h1 style="color:red;">Trade Logic</h>

In [ ]:
def generate_trading_signals(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5'):
    """ Vectorized without iterrows
    Generate buy/sell signals based on moving averages and RSI indicators and save position states.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated signal and position state columns.
    """
    # Dynamically generate signal column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    ma_position_open_col = f'position_open_{ma_name1}_{ma_name2}'
    rsi_position_open_col = f'position_open_{rsi_column}'

    # Initialize signal and position state columns
    candles[ma_signal_column] = 0
    candles[rsi_signal_column] = 0
    candles[ma_position_open_col] = False
    candles[rsi_position_open_col] = False

    # Moving average signal and position state generation
    ma_position_open = False  # Local variable for tracking state
    ma_signals = []
    ma_positions = []
    for ma1, ma2 in zip(candles[ma_name1], candles[ma_name2]):
        if not ma_position_open and ma1 <= ma2:
            ma_position_open = True
            ma_signals.append(1)
        elif ma_position_open and ma1 > ma2:
            ma_position_open = False
            ma_signals.append(0)
        else:
            ma_signals.append(ma_signals[-1] if ma_signals else 0)
        ma_positions.append(ma_position_open)
    candles[ma_signal_column] = ma_signals
    candles[ma_position_open_col] = ma_positions

    # RSI signal and position state generation
    rsi_position_open = False  # Local variable for tracking state
    rsi_signals = []
    rsi_positions = []
    for rsi in candles[rsi_column]:
        if not rsi_position_open and rsi < 50:
            rsi_position_open = True
            rsi_signals.append(1)
        elif rsi_position_open and rsi >= 50:
            rsi_position_open = False
            rsi_signals.append(0)
        else:
            rsi_signals.append(rsi_signals[-1] if rsi_signals else 0)
        rsi_positions.append(rsi_position_open)
    candles[rsi_signal_column] = rsi_signals
    candles[rsi_position_open_col] = rsi_positions

    return candles

def update_position_open(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5'):
    """
    Update the 'ma_position_open' and 'rsi_position_open' columns for a given DataFrame.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing the signals and position columns.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI signal.

    Returns:
    - pd.DataFrame: The updated DataFrame with 'ma_position_open' and 'rsi_position_open' columns.
    """
    # Dynamically generate signal column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    ma_position_open = f'position_open_{ma_name1}_{ma_name2}'
    rsi_position_open = f'position_open_{rsi_column}'
    
    # Update position open columns
    candles[ma_position_open] = candles[ma_signal_column] == 1
    candles[rsi_position_open] = candles[rsi_signal_column] == 1
    
    return candles

def determine_entry_prices(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', ticker_to_tick_size=None, ticker=None):
    """
    Determine entry prices for MA and RSI strategies based on signals.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated entry price columns.
    """
    # Dynamically generate column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}'
    rsi_entry_price = f'entry_price_{rsi_column}'

    # Initialize entry price columns
    candles[ma_entry_price] = None
    candles[rsi_entry_price] = None

    # Get tick size
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Moving Average Strategy
    ma_signals = candles[ma_signal_column]
    ma_close_prices = candles['close']
    ma_entry_mask = (ma_signals == 1) & (ma_signals.shift(1) != 1)
    candles.loc[ma_entry_mask, ma_entry_price] = ma_close_prices[ma_entry_mask] + tick_size

    # RSI Strategy
    rsi_signals = candles[rsi_signal_column]
    rsi_entry_mask = (rsi_signals == 1) & (rsi_signals.shift(1) != 1)
    candles.loc[rsi_entry_mask, rsi_entry_price] = ma_close_prices[rsi_entry_mask] + tick_size

    return candles

def determine_exit_prices(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', ticker_to_tick_size=None, ticker=None):
    """
    Determine exit prices for MA and RSI strategies based on signals.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated exit price columns.
    """
    # Dynamically generate column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}'
    rsi_exit_price = f'exit_price_{rsi_column}'

    # Initialize exit price columns
    candles[ma_exit_price] = None
    candles[rsi_exit_price] = None

    # Get tick size
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Moving Average Strategy
    ma_signals = candles[ma_signal_column]
    ma_close_prices = candles['close']
    ma_exit_mask = (ma_signals == 0) & (ma_signals.shift(1) == 1)
    candles.loc[ma_exit_mask, ma_exit_price] = ma_close_prices[ma_exit_mask] - tick_size

    # RSI Strategy
    rsi_signals = candles[rsi_signal_column]
    rsi_exit_mask = (rsi_signals == 0) & (rsi_signals.shift(1) == 1)
    candles.loc[rsi_exit_mask, rsi_exit_price] = ma_close_prices[rsi_exit_mask] - tick_size

    return candles

def calculate_stop_losses(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5'):
    """
    Dynamically calculate stop loss levels for MA and RSI strategies and ensure they persist while positions are open.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss columns.
    """
    # Dynamically generate column names
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}'
    rsi_entry_price = f'entry_price_{rsi_column}'
    rsi_exit_price = f'exit_price_{rsi_column}'
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}'
    stop_loss_rsi = f'stop_loss_{rsi_column}'

    # Initialize stop loss columns
    candles[stop_loss_ma] = None
    candles[stop_loss_rsi] = None

    # Moving Average Stop Loss
    ma_entry_mask = candles[ma_entry_price].notnull()
    ma_exit_mask = candles[ma_exit_price].notnull()
    
    # Set stop loss where positions open
    candles.loc[ma_entry_mask, stop_loss_ma] = candles[ma_entry_price] - candles['candle_span_max']

    # Reset stop loss and close position where positions close
    candles.loc[ma_exit_mask, stop_loss_ma] = None

    # RSI Stop Loss
    rsi_entry_mask = candles[rsi_entry_price].notnull()
    rsi_exit_mask = candles[rsi_exit_price].notnull()
    
    # Set stop loss where positions open
    candles.loc[rsi_entry_mask, stop_loss_rsi] = candles[rsi_entry_price] - candles['candle_span_max']

    # Reset stop loss and close position where positions close
    candles.loc[rsi_exit_mask, stop_loss_rsi] = None

    # Forward-fill stop loss for both strategies
    candles[stop_loss_ma] = candles[stop_loss_ma].ffill()
    candles[stop_loss_rsi] = candles[stop_loss_rsi].ffill()

    return candles

def track_stop_loss_hits(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', ticker_to_tick_size=None, ticker=None): # Tick size is not being used, address this here and in the function calling
    """
    Track whether stop losses have been hit for MA and RSI strategies and update dynamically named columns.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss hit flags.
    """
    # Dynamically generate column names
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}'
    stop_loss_rsi = f'stop_loss_{rsi_column}'
    ma_stop_loss_hit = f'stop_loss_hit_{ma_name1}_{ma_name2}'
    rsi_stop_loss_hit = f'stop_loss_hit_{rsi_column}'
    ma_position_open = f'position_open_{ma_name1}_{ma_name2}'
    rsi_position_open = f'position_open_{rsi_column}'
    
    # Initialize stop loss hit columns
    candles[ma_stop_loss_hit] = False
    candles[rsi_stop_loss_hit] = False

    # Get tick size
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Ensure stop loss values are numerical (convert None to NaN)
    candles[stop_loss_ma] = candles[stop_loss_ma].fillna(float('inf'))
    candles[stop_loss_rsi] = candles[stop_loss_rsi].fillna(float('inf'))

    # Moving Average Stop Loss Hit Logic
    ma_hit_condition = (candles[stop_loss_ma].notnull()) & (candles['close'] <= candles[stop_loss_ma]) & candles[ma_position_open]
    candles.loc[ma_hit_condition, ma_stop_loss_hit] = True

    # RSI Stop Loss Hit Logic
    rsi_hit_condition = (candles[stop_loss_rsi].notnull()) & (candles['close'] <= candles[stop_loss_rsi]) & candles[rsi_position_open]
    candles.loc[rsi_hit_condition, rsi_stop_loss_hit] = True

    return candles

def adjust_signals_for_stop_loss(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5'):
    """
    Adjust MA and RSI signals to 0 where stop loss has been hit.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.

    Returns:
    - pd.DataFrame: The updated DataFrame with adjusted signals.
    """
    # Dynamically generate column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    ma_stop_loss_hit_column = f'stop_loss_hit_{ma_name1}_{ma_name2}'
    rsi_stop_loss_hit_column = f'stop_loss_hit_{rsi_column}'

    # Adjust MA and RSI signals where stop loss has been hit
    candles.loc[candles[ma_stop_loss_hit_column], ma_signal_column] = 0
    candles.loc[candles[rsi_stop_loss_hit_column], rsi_signal_column] = 0

    return candles

def update_stop_loss(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5'):
    """
    Dynamically set stop loss columns to NaN where corresponding signal columns are 0.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss columns modified.
    """
    # Dynamically generate column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}'
    rsi_signal_column = f'signal_{rsi_column}'
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}'
    stop_loss_rsi = f'stop_loss_{rsi_column}'

    # Update stop loss columns to NaN where signals are 0
    candles.loc[candles[ma_signal_column] == 0, stop_loss_ma] = float('nan')
    candles.loc[candles[rsi_signal_column] == 0, stop_loss_rsi] = float('nan')

    return candles

def calculate_profit_loss(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', contract_multiplier=1, trade_commission=1.5):
    """ 
    Dynamically calculate profit and loss based on entry and exit price columns, including cumulative commission costs.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data with entry and exit price columns.
    - multiplier (float): The multiplier for PnL calculation (e.g., contract size).
    - trade_commission (float): The commission cost per trade.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.

    Returns:
    - pd.DataFrame: The DataFrame with dynamically named profit/loss and commission cost columns.
    """
    # Dynamically generate column names
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}'
    rsi_entry_price = f'entry_price_{rsi_column}'
    rsi_exit_price = f'exit_price_{rsi_column}'
    pnl_ma_col = f'pnl_{ma_name1}_{ma_name2}'
    pnl_rsi_col = f'pnl_{rsi_column}'
    cum_pnl_ma_col = f'cum_{pnl_ma_col}'
    cum_pnl_rsi_col = f'cum_{pnl_rsi_col}'
    cum_pnl_all_col = f'cum_pnl_all_{ma_name1}_{ma_name2}_{rsi_column}'
    ma_commission_col = f'commission_cost_{ma_name1}_{ma_name2}'
    rsi_commission_col = f'commission_cost_{rsi_column}'

    # Initialize PnL and commission columns
    candles[pnl_ma_col] = 0.0
    candles[pnl_rsi_col] = 0.0
    candles[ma_commission_col] = 0.0
    candles[rsi_commission_col] = 0.0

    # Moving Average Strategy PnL and Commission Calculation
    ma_entry_indices = candles.index[candles[ma_entry_price].notnull()]
    ma_exit_indices = candles.index[candles[ma_exit_price].notnull()]

    # Pair up entry and exit prices
    valid_pairs_ma = min(len(ma_entry_indices), len(ma_exit_indices))
    ma_entry_prices = candles.loc[ma_entry_indices[:valid_pairs_ma], ma_entry_price].values
    ma_exit_prices = candles.loc[ma_exit_indices[:valid_pairs_ma], ma_exit_price].values

    # Calculate commission costs for MA strategy
    candles[ma_commission_col] = candles[ma_entry_price].notna().astype(int) * trade_commission + \
                                 candles[ma_exit_price].notna().astype(int) * trade_commission
    candles[ma_commission_col] = candles[ma_commission_col].cumsum()  # Accumulate commission costs

    # Calculate PnL for MA strategy
    ma_pnl = (ma_exit_prices - ma_entry_prices) * contract_multiplier
    candles.loc[ma_exit_indices[:valid_pairs_ma], pnl_ma_col] = ma_pnl

    # RSI Strategy PnL and Commission Calculation
    rsi_entry_indices = candles.index[candles[rsi_entry_price].notnull()]
    rsi_exit_indices = candles.index[candles[rsi_exit_price].notnull()]

    # Pair up entry and exit prices
    valid_pairs_rsi = min(len(rsi_entry_indices), len(rsi_exit_indices))
    rsi_entry_prices = candles.loc[rsi_entry_indices[:valid_pairs_rsi], rsi_entry_price].values
    rsi_exit_prices = candles.loc[rsi_exit_indices[:valid_pairs_rsi], rsi_exit_price].values

    # Calculate commission costs for RSI strategy
    candles[rsi_commission_col] = candles[rsi_entry_price].notna().astype(int) * trade_commission + \
                                  candles[rsi_exit_price].notna().astype(int) * trade_commission
    candles[rsi_commission_col] = candles[rsi_commission_col].cumsum()  # Accumulate commission costs

    # Calculate PnL for RSI strategy
    rsi_pnl = (rsi_exit_prices - rsi_entry_prices) * contract_multiplier
    candles.loc[rsi_exit_indices[:valid_pairs_rsi], pnl_rsi_col] = rsi_pnl

    # Calculate cumulative PnL for both strategies
    candles[cum_pnl_ma_col] = candles[pnl_ma_col].cumsum()
    candles[cum_pnl_rsi_col] = candles[pnl_rsi_col].cumsum()

    # Calculate combined cumulative PnL
    candles[cum_pnl_all_col] = candles[cum_pnl_ma_col] + candles[cum_pnl_rsi_col]

    return candles

In [ ]:
# # Initialize a variable to track if a position is open (market neutral) for function `trade_logic`
# position_open = False  # False means market neutral, True means a trade is active
# entry_price = None  # Store the entry price when a trade is opened

# # Initialize a variable to track if a position is open (market neutral) for function `trade_logic`
# sim_position_open = False  # False means market neutral, True means a trade is active
# sim_entry_price = None  # Store the entry price when a trade is opened

# def trade_logic(candles, buy_order=None, sell_order=None):
#     """
#     Function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.
#     - buy_order (dict): The buy order configuration (market order).
#     - sell_order (dict): The sell order configuration (market order).
#     """

#     global position_open, entry_price  # Ensure access to global variables

#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Check if it's the start of a new minute (second == 00)
#     if current_second == 0 or 1:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         previous_signal = candles.iloc[-2]['signal']

#         if previous_signal == 1:
#             # If signal is 1 and no position is open, go long (buy)
#             if not position_open:
#                 # Place buy order
#                 resp = client.order_place(account_hash, buy_order)
#                 print(f"Buy order placed. Response: {resp}")
                
#                 # Mark position as open and record entry price
#                 position_open = True
#                 entry_price = candles.iloc[-2]['close']
#                 print(f"Bought at {entry_price}")
#             else:
#                 print("Position already open, no action taken.")

#         elif previous_signal == 0 or previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, close the position (sell)
#             if position_open:
#                 # Place sell order
#                 resp = client.order_place(account_hash, sell_order)
#                 print(f"Sell order placed. Response: {resp}")
                
#                 # Close position
#                 position_open = False
#                 print(f"Sold at {candles.iloc[-2]['close']}")
#             else:
#                 print("No position open, no action taken.")

# def trade_simulator(candles):
#     """
#     Simulated function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.

#     The function updates the 'sim_signal', 'sim_entry', and 'sim_exit' columns in the DataFrame.
#     """

#     global sim_position_open, sim_entry_price  # Ensure access to global variables

#     # Initialize the new columns if they don't exist
#     if 'sim_signal' not in candles.columns:
#         candles['sim_signal'] = None
#     if 'sim_entry' not in candles.columns:
#         candles['sim_entry'] = None
#     if 'sim_exit' not in candles.columns:
#         candles['sim_exit'] = None

#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Check if it's the start of a new minute (second == 00)
#     if current_second == 0 or 1:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         sim_previous_signal = candles.iloc[-2]['signal']

#         if sim_previous_signal == 1:
#             # If signal is 1 and no position is open, simulate going long (buy)
#             if not sim_position_open:
#                 # Simulate buy entry
#                 sim_entry_price = candles.iloc[-2]['close']
#                 sim_position_open = True
                
#                 # Record simulated signal and entry price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = 1
#                 candles.at[candles.index[-2], 'sim_entry'] = sim_entry_price

#                 print(f"Simulated buy at {sim_entry_price}")

#             else:
#                 print("Position already open, no action taken.")

#         elif sim_previous_signal == 0 or sim_previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, simulate closing the position (sell)
#             if sim_position_open:
#                 # Simulate sell exit
#                 sim_exit_price = candles.iloc[-2]['close']
#                 sim_position_open = False

#                 # Record simulated signal and exit price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = -1
#                 candles.at[candles.index[-2], 'sim_exit'] = sim_exit_price

#                 print(f"Simulated sell at {sim_exit_price}")
#             else:
#                 print("No position open, no action taken.")

# def trade_simulator(candles):
#     """
#     Simulated function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.

#     The function updates the 'sim_signal' and 'sim_price' columns in the DataFrame.
#     """

#     global sim_position_open, sim_entry_price  # Ensure access to global variables

#     current_time = datetime.now(pytz.timezone('US/Eastern'))
#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour    
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Initialize the new columns if they don't exist
#     if 'sim_signal' not in candles.columns:
#         candles['sim_signal'] = None
#     if 'sim_price' not in candles.columns:
#         candles['sim_price'] = None

#     # Check if it's the start of a new minute (second == 00)
#     if current_time.second == 0:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         sim_previous_signal = candles.iloc[-2]['signal']

#         if sim_previous_signal == 1:
#             # If signal is 1 and no position is open, simulate going long (buy)
#             if not sim_position_open:
#                 # Simulate buy entry
#                 sim_entry_price = candles.iloc[-2]['close']
#                 sim_position_open = True
                
#                 # Record simulated signal and entry price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = 1
#                 candles.at[candles.index[-2], 'sim_price'] = sim_entry_price

#                 print(f"Simulated buy at {sim_entry_price}")

#             else:
#                 print("Position already open, no action taken.")

#         elif sim_previous_signal == 0 or sim_previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, simulate closing the position (sell)
#             if sim_position_open:
#                 # Simulate sell exit
#                 sim_exit_price = candles.iloc[-2]['close']
#                 sim_position_open = False

#                 # Record simulated signal and exit price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = -1
#                 candles.at[candles.index[-2], 'sim_price'] = sim_exit_price

#                 print(f"Simulated sell at {sim_exit_price}")
#             else:
#                 print("No position open, no action taken.")

# <h1 style="color:red;"><b>The Data Handler<b></h1>